In [85]:
# # create empty DataFrame df_picks and append row
# my_cols = ['date', 'days_lookback', 'syms_freq', 'symbols']
# # Creating Empty DataFrame and Storing it in variable df_picks
# df_model_top_picks = pd.DataFrame(columns=my_cols)
# df_model_top_picks

In [86]:
import pandas as pd
from itertools import product
from ast import literal_eval
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.width', 900)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# fp_df_picks  = f'df_picks'  # stock picks by criteria: CAGR/UI, CAGR/rtn_std, rtd/UI
fp_df_model_top_picks = 'df_model_top_picks'  # top stock picks from model developed by back test
fp_df_close_clean = 'df_close_clean'  # historic stocks' closing price

verbose = True
# verbose = False

In [87]:
def get_NYSE_date_n(date, dates_NYSE, n, verbose=False):

  if date in dates_NYSE:
    idx_last = len(dates_NYSE) - 1  # last index of dates_NYSE
    idx_date = dates_NYSE.get_loc(date)  # index of date
    idx_date_n = idx_date + n  # index of date_n
    if verbose:
      print(f"date: {date} is in dates_NYSE,  date's position in dates_NYSE is: {idx_date} of {idx_last},  n: {n},  idx_date_n: {idx_date_n},")
    if 0 <= idx_date_n <= idx_last:  # idx_date_n is within bounds of dates_NYSE
      date_n = dates_NYSE[idx_date_n]
      if verbose:      
        print(f'idx_date_n: {idx_date_n} is wiithin bounds of dates_NYSE ("0 to {idx_last}), date_n: {date_n}"\n')
    else:  # idx_date_n is out-of-bounds of dates_NYSE
      date_n = None   
      if verbose:  
        print(f'idx_date_n: {idx_date_n} is out-of-bounds of dates_NYSE ("0 to {idx_last})"\n')
  else:  # date not in dates_NYSE
      date_n = None  
      if verbose:  
        print(f'date: {date} is not in dates_NYSE\n')

  return date_n  

In [88]:
def get_portf_buy(df_picks_close, date, str_symbols, portf_target, verbose=False):
    import numpy as np
    from ast import literal_eval    
    l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
    # array of closing prices corresponding to symbols in l_syms    
    ar_price = df_picks_close.loc[date][l_syms].values  
    sym_cnt = len(l_syms)  # number of symbols
    amt_per_sym = portf_target / sym_cnt  # target dollar investment in each symbol
    ar_sym_share = np.floor(amt_per_sym / ar_price)  # array of shares for each symbol
    ar_sym_amt = ar_price * ar_sym_share  # array of actual dollar amount invested in each symbol
    portf_value = sum(ar_sym_amt)  # total actual portfolio value
    if verbose:
        # print(date, l_syms, ar_price, ar_sym_share, ar_sym_amt, portf_value)    
        print(f'{date = }, {l_syms = }, {ar_price = }, {ar_sym_share = }, {ar_sym_amt = }, {portf_value = }')            
        print(f'{date} {portf_value = }')
    return date, l_syms, ar_price, ar_sym_share, ar_sym_amt, portf_value

In [89]:
def get_SPY_buy(df_picks_close, date, portf_target, symbol="SPY", verbose=False):   
    import numpy as np
    price = df_picks_close.loc[date][symbol]    
    share = np.floor(portf_target / price)  # array of shares for each symbol
    value = price * share  # array of actual dollar amount invested in each symbol
    if verbose:
        # print(date, symbol, price, share, value)
        print(f'{date = }, {symbol = }, {price = }, {share = }, {value = }')                        
        print(f'{date} {symbol} {value = }')
    return date, price, share, value

### TODO: 2023-12-14, Replace dates_NYSE with dates in df_picks_close, no start_date or end_date needed

In [90]:
df_picks_close = pickle_load(path_data_dump, fp_df_close_clean)
df_picks_mp = pickle_load(path_data_dump, fp_df_model_top_picks)
# df_picks_mp = df_model_top_picks.copy()

In [91]:
dates_sorted = sorted(df_picks_mp.date.tolist())
date_start_picks_mp = dates_sorted[0]
date_end_picks_mp = dates_sorted[-1]
print(date_start_picks_mp, date_end_picks_mp) 

2023-03-15 2023-12-15


In [92]:
# Create a boolean mask for rows between date1 and date2 (inclusive)
mask = (df_picks_close.index >= date_start_picks_mp) & (df_picks_close.index <= date_end_picks_mp)

# Select rows using the mask
df_picks_close = df_picks_close.loc[mask]
df_picks_close

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABR,ABT,ACGL,ACHC,ACIW,ACLS,ACRS,ADBE,...,XRX,XYL,YELP,YUM,YUMC,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-03-15,133.288239,39.372566,13.86,119.255257,152.371918,32.977055,149.485596,10.526908,96.316017,62.740002,68.459999,22.700001,123.860001,7.990,333.609985,...,14.261086,95.604477,29.170000,125.356514,59.979794,26.028101,124.543419,287.739990,73.779999,39.919998,29.475050,25.606165,162.472290,19.500000,20.806547
2023-03-16,136.062805,40.342766,14.12,119.509888,155.220352,33.193268,150.688766,10.499517,97.527351,66.070000,70.230003,25.780001,129.960007,8.120,353.290009,...,14.451233,96.208496,29.780001,126.273537,60.515686,26.922306,124.712440,294.929993,74.510002,40.849998,30.818754,27.639652,165.193893,19.250000,20.796629
2023-03-17,132.442932,38.758762,13.98,116.934204,154.373779,32.883053,149.640839,10.216486,95.538002,63.150002,69.459999,26.940001,128.220001,7.890,358.140015,...,13.899805,93.802299,29.330000,125.080414,60.257668,27.720362,123.598854,288.709991,74.129997,39.810001,28.736017,28.083862,163.366241,18.540001,20.360270
2023-03-20,133.795395,39.877464,13.96,116.229065,156.764084,33.024059,151.484421,10.791677,96.384964,65.610001,70.449997,26.150000,131.509995,7.950,362.880005,...,14.118475,96.693703,29.690001,127.082108,60.485916,28.028044,125.607292,290.839996,74.650002,39.830002,28.966366,27.659395,164.707199,18.110001,20.657787
2023-03-21,136.251740,41.491177,14.37,117.316147,158.636505,33.334270,152.115128,10.873848,96.837975,67.209999,70.959999,26.200001,133.449997,8.240,374.220001,...,14.546309,97.555176,30.059999,126.756706,60.704235,27.489597,127.058952,294.429993,75.570000,41.939999,30.991514,27.866692,165.124374,18.549999,21.004896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-11,128.970001,24.930000,13.72,57.080002,193.179993,29.820000,151.240005,14.100000,106.220001,79.610001,74.959999,28.000000,125.580002,0.973,625.200012,...,15.170000,107.699997,44.099998,126.629997,39.919998,36.939999,117.980003,239.649994,63.730000,43.820000,38.630001,21.020000,189.460007,18.969999,29.610001
2023-12-12,128.789993,24.049999,14.04,56.250000,194.710007,30.559999,153.240005,14.190000,106.680000,80.279999,75.910004,28.320000,126.190002,1.010,633.659973,...,15.710000,107.940002,44.480000,128.070007,40.320000,39.029999,117.449997,239.380005,62.560001,45.419998,38.130001,21.020000,191.470001,18.570000,29.230000
2023-12-13,133.740005,25.990000,14.09,60.830002,197.960007,30.809999,154.300003,14.880000,107.250000,78.379997,78.959999,29.240000,131.500000,1.090,624.260010,...,16.709999,109.180000,44.490002,130.929993,39.750000,38.500000,117.900002,244.429993,64.720001,50.279999,41.820000,20.400000,197.410004,19.670000,29.320000


In [93]:
# import pandas_market_calendars as mcal
# nyse = mcal.get_calendar('NYSE')
# dates_NYSE_2013_2023 = nyse.valid_days(start_date='2013-01-01', end_date='2023-12-31').strftime('%Y-%m-%d')
# print(f'len(dates_NYSE_2013_2023): {len(dates_NYSE_2013_2023)}')
# print(f'dates_NYSE_2013_2023[0]:  {dates_NYSE_2013_2023[0]}, dates_NYSE_2013_2023[-1]: {dates_NYSE_2013_2023[-1]}')
# print(f'{dates_NYSE_2013_2023}') 

len(dates_NYSE_2013_2023): 2768
dates_NYSE_2013_2023[0]:  2013-01-02, dates_NYSE_2013_2023[-1]: 2023-12-29
Index(['2013-01-02', '2013-01-03', '2013-01-04', '2013-01-07', '2013-01-08', '2013-01-09', '2013-01-10', '2013-01-11', '2013-01-14', '2013-01-15',
       ...
       '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20', '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27', '2023-12-28', '2023-12-29'], dtype='object', length=2768)


In [94]:
dates_NYSE_2013_2023 = df_picks_close.index

In [95]:
# from ast import literal_eval    
# l_syms = literal_eval(syms)  # convert list stored as str back to list

# for _sym in l_syms:
#   ans = _sym in df_picks_close.columns
#   print(_sym, ans)

In [96]:
def all_in_list(list1, list2):
  """
  Checks if all items in list1 are also in list2.

  Args:
    list1: A list of items.
    list2: Another list of items.

  Returns:
    True if all items in list1 are in list2, False otherwise.
  """
  return set(list1).issubset(set(list2))

In [97]:
def any_not_in_list(list1, list2):
  """
  Checks if any items in list1 are not in list2.

  Args:
    list1: A list of items.
    list2: Another list of items.

  Returns:
    True if any item in list1 is not in list2, False if all are present.
  """
  return bool(set(list1) - set(list2))

In [98]:
# l_syms = ['FTSM', 'SHV']
l_syms = ['GPS', 'SHV']
# l_syms = ['ISEE', 'SHV']
# l_syms = ['SHV', 'ISEE']
# l_syms = ['ISEE', 'ISEE']
# l_syms = ['SHV', 'SHV']
# l_syms = ['SHV', 'ISEE', 'SHV']
# l_syms = ['AAPL', 'NVDA']
# l_syms = ['AAPL', 'NVDA', 'SHV', 'AAPL', 'NVDA', 'SHV']
# ans = all_in_list(l_syms, df_picks_close.columns)
ans = any_not_in_list(l_syms, df_picks_close.columns)
print(l_syms, ans)

['GPS', 'SHV'] False


In [99]:
print(any_not_in_list(l_syms, df_picks_close.columns))
print(l_syms)

False
['GPS', 'SHV']


In [100]:
False or False
False or True
# True or True

True

In [101]:
import numpy as np
# from ast import literal_eval
z_date_syms = zip(df_picks_mp.date, df_picks_mp.symbols)

idx_close = df_picks_close.index.strftime('%Y-%m-%d')
symbols_df_picks_close = df_picks_close.columns  # symbols in df_picks_close

n = 1  # n=1 is the buy date
# n = 4  # n=4 is the sell date

date_buy = []  # buy date of portfolio
shares_syms = []  # lists of shares of each symbol brought on date_buy
value_portf = [] # list of porfolio value on date_buy
shares_SPY = []  # list of shares of SPY brought on date_buy
value_SPY = []  # list of value of SPY shares on date_buy 
for date, syms in z_date_syms:
    next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=verbose)

    if next_date_n in idx_close:
        close_date_n = next_date_n
        print(f'date: {date},  n: {n},  next_date_n: {next_date_n} is in df_picks_close')
        print(f'picks for {date}: {syms}')      
    else:
        close_date_n = None
        print(f'date: {date},  n: {n},  next_date_n: {next_date_n} is not in df_picks_close')
        print(f'picks for {date}: {syms}')        

    l_syms = literal_eval(syms)  # convert list stored as str back to list
    # True if symbol(s) in l_syms is not a column in df_picks_close
    sym_not_in_df_picks_close = any_not_in_list(l_syms, symbols_df_picks_close)

    if close_date_n is None or sym_not_in_df_picks_close:
        p_date = None  # portf. buy date
        p_l_syms = None  # portf. list of symbols
        p_ar_price = None  # portf. list of symbols' close
        p_ar_sym_share = None  # portf. list of symbols' share
        p_ar_sym_amt = None  # portf. list of symbols' dollar value 
        p_portf_value = None  # portf. value is none, data not available in df_picks_close
        
        s_date = None  # SPY buy date
        s_price = None # SPY list of symbols' close
        s_share = None # SPY list of symbols' share
        s_value = None  # SPY value is none, data not available in df_picks_close

        date_buy.append(p_date)
        shares_syms.append(p_l_syms)
        value_portf.append(p_portf_value)
        shares_SPY.append(s_share)
        value_SPY.append(s_value)    
        
        print(f"No data for next_date_n {next_date_n}, pick's portf value = None")
        print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

    else:    
        p_date, p_l_syms, p_ar_price, p_ar_sym_share, p_ar_sym_amt, p_portf_value = \
            get_portf_buy(df_picks_close, next_date_n, syms, portf_target=1000, verbose=verbose)
        s_date, s_price, s_share, s_value = \
            get_SPY_buy(df_picks_close, next_date_n, portf_target=1000, verbose=verbose)
                
        date_buy.append(p_date)
        shares_syms.append(p_ar_sym_share)
        value_portf.append(p_portf_value)
        shares_SPY.append(s_share)
        value_SPY.append(s_value)

        print(f"next_date_n pick's portf value = {p_portf_value}")
        print(f'next_date_n SPY portf value =    {s_value}')

    print('='*20, '\n')       
    

date: 2023-12-15 is in dates_NYSE,  date's position in dates_NYSE is: 191 of 191,  n: 1,  idx_date_n: 192,
idx_date_n: 192 is out-of-bounds of dates_NYSE ("0 to 191)"

date: 2023-12-15,  n: 1,  next_date_n: None is not in df_picks_close
picks for 2023-12-15: ['SHV']
No data for next_date_n None, pick's portf value = None
No data for next_date_n None, SPY portf value =    None

date: 2023-12-13 is in dates_NYSE,  date's position in dates_NYSE is: 189 of 191,  n: 1,  idx_date_n: 190,
idx_date_n: 190 is wiithin bounds of dates_NYSE ("0 to 191), date_n: 2023-12-14 00:00:00"

date: 2023-12-13,  n: 1,  next_date_n: 2023-12-14 00:00:00 is not in df_picks_close
picks for 2023-12-13: ['GPS', 'SHV']
No data for next_date_n 2023-12-14 00:00:00, pick's portf value = None
No data for next_date_n 2023-12-14 00:00:00, SPY portf value =    None

date: 2023-12-12 is in dates_NYSE,  date's position in dates_NYSE is: 188 of 191,  n: 1,  idx_date_n: 189,
idx_date_n: 189 is wiithin bounds of dates_NYSE ("0

#### TODO: 2023-12-17 skip syms that are not in df_picks_close, these symbols are no-longer listed

#### TODO: 2023-07-15 only evaluate performance of df_modelpicks, after evaluation, append results to df_modelpicks_results 

#### 2023-07-11 got KeyError: "['ISEE'] not in index". ISEE was in past model picks, but since then ISEE was acquired. The symbol was dropped from df_close_clean. Current program recalculate the performance of all the past picks. Rewrite program such that performance of past picks with win records are not recalculated.

In [102]:
# import numpy as np
# # from ast import literal_eval
# z_date_syms = zip(df_picks_mp.date, df_picks_mp.symbols)

# idx_close = df_picks_close.index.strftime('%Y-%m-%d')

# n = 1  # n=1 is the buy date
# # n = 4  # n=4 is the sell date

# date_buy = []  # buy date of portfolio
# shares_syms = []  # lists of shares of each symbol brought on date_buy
# value_portf = [] # list of porfolio value on date_buy
# shares_SPY = []  # list of shares of SPY brought on date_buy
# value_SPY = []  # list of value of SPY shares on date_buy 
# for date, syms in z_date_syms:
#     next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=verbose)

#     if next_date_n in idx_close:
#         close_date_n = next_date_n
#         print(f'date: {date},  n: {n},  next_date_n: {next_date_n} is in df_picks_close')
#         print(f'picks for {date}: {syms}')      
#     else:
#         close_date_n = None
#         print(f'date: {date},  n: {n},  next_date_n: {next_date_n} is not in df_picks_close')
#         print(f'picks for {date}: {syms}')        

#     if close_date_n is None:
#         p_date = None  # portf. buy date
#         p_l_syms = None  # portf. list of symbols
#         p_ar_price = None  # portf. list of symbols' close
#         p_ar_sym_share = None  # portf. list of symbols' share
#         p_ar_sym_amt = None  # portf. list of symbols' dollar value 
#         p_portf_value = None  # portf. value is none, data not available in df_picks_close
        
#         s_date = None  # SPY buy date
#         s_price = None # SPY list of symbols' close
#         s_share = None # SPY list of symbols' share
#         s_value = None  # SPY value is none, data not available in df_picks_close

#         date_buy.append(p_date)
#         shares_syms.append(p_l_syms)
#         value_portf.append(p_portf_value)
#         shares_SPY.append(s_share)
#         value_SPY.append(s_value)    
        
#         print(f"No data for next_date_n {next_date_n}, pick's portf value = None")
#         print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

#     else:    
#         p_date, p_l_syms, p_ar_price, p_ar_sym_share, p_ar_sym_amt, p_portf_value = \
#             get_portf_buy(df_picks_close, next_date_n, syms, portf_target=1000, verbose=verbose)
#         s_date, s_price, s_share, s_value = \
#             get_SPY_buy(df_picks_close, next_date_n, portf_target=1000, verbose=verbose)
                
#         date_buy.append(p_date)
#         shares_syms.append(p_ar_sym_share)
#         value_portf.append(p_portf_value)
#         shares_SPY.append(s_share)
#         value_SPY.append(s_value)

#         print(f"next_date_n pick's portf value = {p_portf_value}")
#         print(f'next_date_n SPY portf value =    {s_value}')

#     print('='*20, '\n')       
    

#### Below cells calculate past performance of model picks

In [103]:
def calc_portf_shares(df_picks_close, date, str_symbols, portf_target):
  # calculate number of shares to buy for symbols in str_symbols to meet port_target
  import numpy as np
  from ast import literal_eval    
  l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
  # array of closing prices corresponding to symbols in l_syms    
  ar_price = df_picks_close.loc[date][l_syms].values  
  sym_cnt = len(l_syms)  # number of symbols
  amt_per_sym = portf_target / sym_cnt  # target dollar investment in each symbol
  ar_shares = np.floor(amt_per_sym / ar_price)  # array of shares for each symbol
  return ar_shares

In [104]:
def calc_portf_value(df_picks_close, date, str_symbols, ar_shares, verbose=False):
    import numpy as np
    from ast import literal_eval    
    l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
    # array of closing prices corresponding to symbols in l_syms    
    ar_price = df_picks_close.loc[date][l_syms].values  
    ar_value = ar_price * ar_shares  # array of actual dollar amount invested in each symbol
    portf_value = sum(ar_value)  # total actual portfolio value
    if verbose:
        print(f'{date = }, {l_syms = }, {ar_price = }, {ar_shares = }, {ar_value = }, {portf_value = }')            
        print(f'{date} {portf_value = }')
    return date, l_syms, ar_price, ar_shares, ar_value, portf_value

In [105]:
def get_next_date_n(df_picks_close, date, n=1):

    """Get the next n(default=1) date from df_picks_close. date has to in df_picks_close.index.

    Edge case: The function won't return the first date in df_picks_close as the
    next trading date, if date is a trading date just before the first date in df_picks_close.
    """

    len_df_picks_close = len(df_picks_close)
    idx_dates = df_picks_close.index.strftime('%Y-%m-%d')  # date index of df_picks_close    

    if date in idx_dates:
        idx_next_date = idx_dates.get_loc(date) + n  # get index position of next date
        print(f'n: {n},  idx_next_date: {idx_next_date}')

        if idx_next_date >= 0:  # if idx_nex_date is negative, stop index loop back from index[0] to index[-1]    
            if idx_next_date >= (len_df_picks_close - n):  # next date's index is within bound of df_picks_close
                next_date = idx_dates[idx_dates.get_loc(date) + n]  # get the next date
                return next_date
        
              
            else:  # idx_next_date is out-of_bounds df_picks_close
                print(f"{date} is the last date in df_picks_close. No data for next {n} day")
                return None    
        
        
        else:  # idx_next_date is negative, no data in df_picks_close
            # no data from df_picks_close
            print(f'idx_next_date: {idx_next_date} is negative, no data for {n} days from {date} in df_picks_close')

    else:     
        # no data from df_picks_close
        print(f'no data for {date} in df_picks_close')
        return None

In [106]:
def is_date_in_close(date, df_picks_close):
  idx_close = df_picks_close.index.strftime('%Y-%m-%d')
  if date in idx_close:
    return date
  else:
    return None

In [107]:
def calc_portf_value_date_buy_(dates_in_days_lookbacks, my_symbols, df_picks_close, portf_target, n, verbose=False):
  
  z_date_syms = zip(dates_in_days_lookbacks, my_symbols)

  date_exec = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date
  value_portf = [] # list of porfolio value on date
  shares_SPY = []  # list of shares of SPY brought on date
  value_SPY = []  # list of value of SPY shares on date 

  for date, syms in z_date_syms:
    next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=False)
    close_date_n = is_date_in_close(next_date_n, df_picks_close)



    l_syms = literal_eval(syms)  # convert list stored as str back to list
    # True if symbol(s) in l_syms is not a column in df_picks_close
    sym_not_in_df_picks_close = any_not_in_list(l_syms, symbols_df_picks_close)

    if close_date_n is None or sym_not_in_df_picks_close:



    # if close_date_n is None:
      p_date = None
      p_ar_shares = None
      p_portf_value = None  # set to None when data are not available in df_picks_close
      SPY_shares = None
      SPY_value = None  # set to None when data are not available in df_picks_close

      if verbose:
        print(f"No data for close_date_n {close_date_n}, pick's portf value = None")
        print(f'No data for close_date_n {close_date_n}, SPY portf value =    None')

    else:    
      p_ar_shares = calc_portf_shares(df_picks_close, close_date_n, syms, portf_target)
      p_date, l_syms, ar_price, ar_shares, ar_value, p_portf_value = \
        calc_portf_value(df_picks_close, close_date_n, syms, p_ar_shares, verbose)

      syms = str(['SPY'])
      SPY_shares = calc_portf_shares(df_picks_close, close_date_n, syms, portf_target)
      date, l_syms, ar_price, ar_shares, ar_value, SPY_value = \
        calc_portf_value(df_picks_close, close_date_n, syms, SPY_shares, verbose)

      if verbose:
        print(f"close_date_n pick's portf value = {p_portf_value}")
        print(f'close_date_n SPY portf value =    {SPY_value}')

    date_exec.append(p_date)
    shares_syms.append(p_ar_shares)
    value_portf.append(p_portf_value)
    shares_SPY.append(SPY_shares)
    value_SPY.append(SPY_value)

    print('='*20, '\n')

  return date_exec, shares_syms, value_portf, shares_SPY, value_SPY       
    

In [108]:
def calc_portf_value_date_buy(dates_in_days_lookbacks, my_symbols, df_picks_close, portf_target, n, verbose=False):
  
  z_date_syms = zip(dates_in_days_lookbacks, my_symbols)

  date_exec = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date
  value_portf = [] # list of porfolio value on date
  shares_SPY = []  # list of shares of SPY brought on date
  value_SPY = []  # list of value of SPY shares on date 

  for date, syms in z_date_syms:
    next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=False)
    close_date_n = is_date_in_close(next_date_n, df_picks_close)

    if close_date_n is None:
      p_date = None
      p_ar_shares = None
      p_portf_value = None  # set to None when data are not available in df_picks_close
      SPY_shares = None
      SPY_value = None  # set to None when data are not available in df_picks_close

      if verbose:
        print(f"No data for close_date_n {close_date_n}, pick's portf value = None")
        print(f'No data for close_date_n {close_date_n}, SPY portf value =    None')

    else:    
      p_ar_shares = calc_portf_shares(df_picks_close, close_date_n, syms, portf_target)
      p_date, l_syms, ar_price, ar_shares, ar_value, p_portf_value = \
        calc_portf_value(df_picks_close, close_date_n, syms, p_ar_shares, verbose)

      syms = str(['SPY'])
      SPY_shares = calc_portf_shares(df_picks_close, close_date_n, syms, portf_target)
      date, l_syms, ar_price, ar_shares, ar_value, SPY_value = \
        calc_portf_value(df_picks_close, close_date_n, syms, SPY_shares, verbose)

      if verbose:
        print(f"close_date_n pick's portf value = {p_portf_value}")
        print(f'close_date_n SPY portf value =    {SPY_value}')

    date_exec.append(p_date)
    shares_syms.append(p_ar_shares)
    value_portf.append(p_portf_value)
    shares_SPY.append(SPY_shares)
    value_SPY.append(SPY_value)

    print('='*20, '\n')

  return date_exec, shares_syms, value_portf, shares_SPY, value_SPY       
    

In [109]:
def calc_portf_value_date_n(dates_in_days_lookbacks, my_symbols, df_picks_close, my_portf_shares, my_SPY_shares, n, verbose=False):
  
  z_date_syms_shares = zip(dates_in_days_lookbacks, my_symbols, my_portf_shares, my_SPY_shares)

  date_exec = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date
  value_portf = [] # list of porfolio value on date
  shares_SPY = []  # list of shares of SPY brought on date
  value_SPY = []  # list of value of SPY shares on date 

  for date, symbols, portf_shares, SPY_shares in z_date_syms_shares:
    next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=False)
    close_date_n = is_date_in_close(next_date_n, df_picks_close)

    if close_date_n is None:
      p_date_exec = None
      p_ar_shares = None
      p_value_portf = None  # set to None when data are not available in df_picks_close
      SPY_ar_shares = None
      SPY_value_portf = None # set to None when data are not available in df_picks_close

      if verbose:
        print(f"No data for close_date_n {close_date_n}, pick's portf value = None")
        # print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

    else: 
      if portf_shares is None:
        p_date_exec = None
        p_ar_shares = None
        p_value_portf = None
        SPY_ar_shares = None
        SPY_value_portf = None

      else:  
        p_date_exec, p_ar_syms, p_ar_price, p_ar_shares, p_ar_value, p_value_portf = \
          calc_portf_value(df_picks_close, close_date_n, symbols, portf_shares, verbose)
        
        SPY = str(['SPY'])
        SPY_date_exec, SPY_ar_syms, SPY_ar_price, SPY_ar_shares, SPY_ar_value, SPY_value_portf = \
          calc_portf_value(df_picks_close, close_date_n, SPY, SPY_shares, verbose) 

        if verbose:
          print(f"next_date_n pick's portf value = {p_value_portf}")
          print(f'next_date_n SPY portf value =    {SPY_value_portf}')

    date_exec.append(p_date_exec)
    shares_syms.append(p_ar_shares)
    value_portf.append(p_value_portf)
    shares_SPY.append(SPY_ar_shares)
    value_SPY.append(SPY_value_portf)

    print('='*20, '\n')

  return date_exec, shares_syms, value_portf, shares_SPY, value_SPY  

In [110]:
df_picks_mp

,date,days_lookback,syms_freq,symbols
0,2023-12-15,"[30, 60, 120]",sym_freq_8,['SHV']
1,2023-12-13,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']"
2,2023-12-12,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']"
3,2023-12-11,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']"
4,2023-12-08,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']"
...,...,...,...,...
285,2023-03-20,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']"
286,2023-03-17,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']"
287,2023-03-17,"[30, 60, 120]",sym_freq_8,['GE']
288,2023-03-16,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']"


In [111]:
df_picks_mp.date

0      2023-12-15
1      2023-12-13
2      2023-12-12
3      2023-12-11
4      2023-12-08
          ...    
285    2023-03-20
286    2023-03-17
287    2023-03-17
288    2023-03-16
289    2023-03-15
Name: date, Length: 290, dtype: object

In [112]:
df_picks_mp.symbols

0              ['SHV']
1       ['GPS', 'SHV']
2       ['GPS', 'SHV']
3       ['GPS', 'SHV']
4       ['GPS', 'SHV']
            ...       
285    ['FTSM', 'SHV']
286    ['FTSM', 'SHV']
287             ['GE']
288    ['FTSM', 'SHV']
289    ['FTSM', 'SHV']
Name: symbols, Length: 290, dtype: object

In [113]:
df_picks_close

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABR,ABT,ACGL,ACHC,ACIW,ACLS,ACRS,ADBE,...,XRX,XYL,YELP,YUM,YUMC,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-03-15,133.288239,39.372566,13.86,119.255257,152.371918,32.977055,149.485596,10.526908,96.316017,62.740002,68.459999,22.700001,123.860001,7.990,333.609985,...,14.261086,95.604477,29.170000,125.356514,59.979794,26.028101,124.543419,287.739990,73.779999,39.919998,29.475050,25.606165,162.472290,19.500000,20.806547
2023-03-16,136.062805,40.342766,14.12,119.509888,155.220352,33.193268,150.688766,10.499517,97.527351,66.070000,70.230003,25.780001,129.960007,8.120,353.290009,...,14.451233,96.208496,29.780001,126.273537,60.515686,26.922306,124.712440,294.929993,74.510002,40.849998,30.818754,27.639652,165.193893,19.250000,20.796629
2023-03-17,132.442932,38.758762,13.98,116.934204,154.373779,32.883053,149.640839,10.216486,95.538002,63.150002,69.459999,26.940001,128.220001,7.890,358.140015,...,13.899805,93.802299,29.330000,125.080414,60.257668,27.720362,123.598854,288.709991,74.129997,39.810001,28.736017,28.083862,163.366241,18.540001,20.360270
2023-03-20,133.795395,39.877464,13.96,116.229065,156.764084,33.024059,151.484421,10.791677,96.384964,65.610001,70.449997,26.150000,131.509995,7.950,362.880005,...,14.118475,96.693703,29.690001,127.082108,60.485916,28.028044,125.607292,290.839996,74.650002,39.830002,28.966366,27.659395,164.707199,18.110001,20.657787
2023-03-21,136.251740,41.491177,14.37,117.316147,158.636505,33.334270,152.115128,10.873848,96.837975,67.209999,70.959999,26.200001,133.449997,8.240,374.220001,...,14.546309,97.555176,30.059999,126.756706,60.704235,27.489597,127.058952,294.429993,75.570000,41.939999,30.991514,27.866692,165.124374,18.549999,21.004896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-11,128.970001,24.930000,13.72,57.080002,193.179993,29.820000,151.240005,14.100000,106.220001,79.610001,74.959999,28.000000,125.580002,0.973,625.200012,...,15.170000,107.699997,44.099998,126.629997,39.919998,36.939999,117.980003,239.649994,63.730000,43.820000,38.630001,21.020000,189.460007,18.969999,29.610001
2023-12-12,128.789993,24.049999,14.04,56.250000,194.710007,30.559999,153.240005,14.190000,106.680000,80.279999,75.910004,28.320000,126.190002,1.010,633.659973,...,15.710000,107.940002,44.480000,128.070007,40.320000,39.029999,117.449997,239.380005,62.560001,45.419998,38.130001,21.020000,191.470001,18.570000,29.230000
2023-12-13,133.740005,25.990000,14.09,60.830002,197.960007,30.809999,154.300003,14.880000,107.250000,78.379997,78.959999,29.240000,131.500000,1.090,624.260010,...,16.709999,109.180000,44.490002,130.929993,39.750000,38.500000,117.900002,244.429993,64.720001,50.279999,41.820000,20.400000,197.410004,19.670000,29.320000


In [114]:
# total target investment for each day's picks, if day's pick has 2 symbols, the function will try to invest $500 for each symbol
portf_target = 1000  
date_buy, shares_syms, value_portf, shares_SPY, value_SPY = \
  calc_portf_value_date_buy_(df_picks_mp.date, df_picks_mp.symbols, df_picks_close, portf_target, n=1, verbose=verbose)

No data for close_date_n None, pick's portf value = None
No data for close_date_n None, SPY portf value =    None

No data for close_date_n None, pick's portf value = None
No data for close_date_n None, SPY portf value =    None

No data for close_date_n None, pick's portf value = None
No data for close_date_n None, SPY portf value =    None

No data for close_date_n None, pick's portf value = None
No data for close_date_n None, SPY portf value =    None

No data for close_date_n None, pick's portf value = None
No data for close_date_n None, SPY portf value =    None

No data for close_date_n None, pick's portf value = None
No data for close_date_n None, SPY portf value =    None

No data for close_date_n None, pick's portf value = None
No data for close_date_n None, SPY portf value =    None

No data for close_date_n None, pick's portf value = None
No data for close_date_n None, SPY portf value =    None

No data for close_date_n None, pick's portf value = None
No data for close_date_

In [115]:
# # total target investment for each day's picks, if day's pick has 2 symbols, the function will try to invest $500 for each symbol
# portf_target = 1000  
# date_buy, shares_syms, value_portf, shares_SPY, value_SPY = \
#   calc_portf_value_date_buy(df_picks_mp.date, df_picks_mp.symbols, df_picks_close, portf_target, n=1, verbose=verbose)

In [116]:
df_picks_mp['date_buy'] = date_buy
df_picks_mp['sh_portf_buy'] = shares_syms
df_picks_mp['$_portf_buy'] = value_portf
df_picks_mp['sh_SPY_buy'] = shares_SPY
df_picks_mp['$_SPY_buy'] = value_SPY
# df_picks_mp.tail()
# df_picks_mp.head()
df_picks_mp

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy
0,2023-12-15,"[30, 60, 120]",sym_freq_8,['SHV'],None,None,None,None,None
1,2023-12-13,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None
2,2023-12-12,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None
3,2023-12-11,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None
4,2023-12-08,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
285,2023-03-20,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",None,None,None,None,None
286,2023-03-17,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",None,None,None,None,None
287,2023-03-17,"[30, 60, 120]",sym_freq_8,['GE'],None,None,None,None,None
288,2023-03-16,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",None,None,None,None,None


In [117]:
date_exec, shares_syms, value_portf, shares_SPY, value_SPY  = \
  calc_portf_value_date_n(df_picks_mp.date, df_picks_mp.symbols, df_picks_close, df_picks_mp.sh_portf_buy, df_picks_mp.sh_SPY_buy, n=4, verbose=verbose)

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for cl

In [118]:
df_picks_mp['date_sell'] = date_exec
df_picks_mp['sh_portf_sell'] = shares_syms
df_picks_mp['$_portf_sell'] = value_portf
df_picks_mp['%_portf_chg'] = (df_picks_mp['$_portf_sell'] / df_picks_mp['$_portf_buy'] - 1) * 100

df_picks_mp['sh_SPY_sell'] = shares_SPY
df_picks_mp['$_SPY_sell'] = value_SPY
df_picks_mp['%_SPY_chg'] = (df_picks_mp['$_SPY_sell'] / df_picks_mp['$_SPY_buy'] - 1) * 100

df_picks_mp['dif_%_chg'] = df_picks_mp['%_portf_chg'] - df_picks_mp['%_SPY_chg']

In [119]:
# # get symbol's close price for that date
# date = '2023-03-16'
# # _date = '2023-03-21'
# # _sym = 'GE'
# # _sym = 'NVDA'
# _sym = 'FTSM'
# # _sym = 'SHV'
# _sym = 'SPY'
# # _c = df_picks_close.loc['2023-04-11']['NVDA']
# _c = df_picks_close.loc[_date][_sym]
# _c

#### dif_%_chg is the percentage change of model icks - percentage change of SPY

In [120]:
df_picks_mp.head(20)

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy,date_sell,sh_portf_sell,$_portf_sell,%_portf_chg,sh_SPY_sell,$_SPY_sell,%_SPY_chg,dif_%_chg
0,2023-12-15,"[30, 60, 120]",sym_freq_8,['SHV'],None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN
1,2023-12-13,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN
2,2023-12-12,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN
3,2023-12-11,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN
4,2023-12-08,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN
5,2023-12-07,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN
6,2023-12-06,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN
7,2023-12-05,"[30, 60, 120]",sym_freq_9,['GPS'],None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN
8,2023-12-05,"[30, 60, 120]",sym_freq_8,"['LRN', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN
9,2023-12-05,"[15, 30, 60, 120]",sym_freq_12,['GPS'],None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN


In [121]:
df_picks_mp['win'] = np.where(df_picks_mp['dif_%_chg']>0,1,0)
df_picks_mp.head(20)

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy,date_sell,sh_portf_sell,$_portf_sell,%_portf_chg,sh_SPY_sell,$_SPY_sell,%_SPY_chg,dif_%_chg,win
0,2023-12-15,"[30, 60, 120]",sym_freq_8,['SHV'],None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
1,2023-12-13,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
2,2023-12-12,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
3,2023-12-11,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
4,2023-12-08,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
5,2023-12-07,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
6,2023-12-06,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
7,2023-12-05,"[30, 60, 120]",sym_freq_9,['GPS'],None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
8,2023-12-05,"[30, 60, 120]",sym_freq_8,"['LRN', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
9,2023-12-05,"[15, 30, 60, 120]",sym_freq_12,['GPS'],None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0


### Model Performance Not Good

In [122]:
wins = df_picks_mp['win'].sum()
attempts = len(df_picks_mp['dif_%_chg'].dropna())

win_rate = wins / attempts
print(f'win_rate: {win_rate:0.6f}, wins: {wins}, attempts: {attempts}')
print(f'sum(df_picksf_%_chg): {df_picks_mp["dif_%_chg"].sum():0.6f}')


win_rate: nan, wins: 0, attempts: 0
sum(df_picksf_%_chg): 0.000000


C:\Users\ping\AppData\Local\Temp\ipykernel_5664\1295536089.py:4: RuntimeWarning: invalid value encountered in scalar divide
  win_rate = wins / attempts


In [123]:
df_picks = df_picks_mp.copy()

In [124]:
df_modelpicks = df_picks[df_picks['dif_%_chg'].isnull()].copy()
pickle_dump(df_modelpicks, path_data_dump, 'df_modelpicks')
_df_picks = pickle_load(path_data_dump, 'df_modelpicks')
_df_picks

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy,date_sell,sh_portf_sell,$_portf_sell,%_portf_chg,sh_SPY_sell,$_SPY_sell,%_SPY_chg,dif_%_chg,win
0,2023-12-15,"[30, 60, 120]",sym_freq_8,['SHV'],None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
1,2023-12-13,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
2,2023-12-12,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
3,2023-12-11,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
4,2023-12-08,"[30, 60, 120]",sym_freq_8,"['GPS', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,2023-03-20,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
286,2023-03-17,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
287,2023-03-17,"[30, 60, 120]",sym_freq_8,['GE'],None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0
288,2023-03-16,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,0


In [125]:
idx_last_row = df_modelpicks.index[-1]
df_modelpicks_results = df_picks.iloc[(idx_last_row + 1)::].copy()
pickle_dump(df_modelpicks_results, path_data_dump, 'df_modelpicks_results')
_df_picks = pickle_load(path_data_dump, 'df_modelpicks_results')
_df_picks

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy,date_sell,sh_portf_sell,$_portf_sell,%_portf_chg,sh_SPY_sell,$_SPY_sell,%_SPY_chg,dif_%_chg,win
